In [ ]:
import righor
import matplotlib.pyplot as plt
import seaborn
import pandas as pd
from tqdm.notebook import tqdm
from collections import Counter
import numpy as np
import os
os.environ["RUST_BACKTRACE"] = "1"

# load the model
igor_model = righor.load_model("human", "trb")
# no error
igor_model.error = righor.ErrorParameters.constant_error(0.0)

In [ ]:
# test generic nucleotide
alignment = righor.AlignmentParameters

alignment = righor.AlignmentParameters()
alignment.left_v_cutoff = 600
alignment.max_error_d = 200
alignment.min_score_v = -20
alignment.min_score_j = 0

my_sequence =     f"GAAGCCCAAGTGACCCAGAACCCAAGATACCTCATCACAGTGACTGGAAAGAAGTTAACAGTGACTTGTTCTCAGAATATGAACCATGAGTATATGTCCTGGTATCGACAAGACCCAGGGCTGGGCTTAAGGCAGATCTACTATTCAATGAATGTTGAGGTGACTGATAAGGGAGATGTTCCTGAAGGGTACAAAGTCTCTCGAAAAGAGAAGAGGAATTTCCCCCTGATCCTGGAGTCGCCCAGCCCCAACCAGACCTCTCTGTACTTCTGTGCCAGCCGACNNGACAGCTAACTATGGCTACACCTTCGGTTCGGGGACCAGGTTAACCGTTGTAG"
result_inference = igor_model.evaluate(my_sequence, align_params=alignment)
print("Generic nucleotide test: ", result_inference.likelihood)

s = 0
result = {}
for a in 'ATGC':
    for b in 'ATGC':
        my_seq  = f"GAAGCCCAAGTGACCCAGAACCCAAGATACCTCATCACAGTGACTGGAAAGAAGTTAACAGTGACTTGTTCTCAGAATATGAACCATGAGTATATGTCCTGGTATCGACAAGACCCAGGGCTGGGCTTAAGGCAGATCTACTATTCAATGAATGTTGAGGTGACTGATAAGGGAGATGTTCCTGAAGGGTACAAAGTCTCTCGAAAAGAGAAGAGGAATTTCCCCCTGATCCTGGAGTCGCCCAGCCCCAACCAGACCTCTCTGTACTTCTGTGCCAGCCGAC{a}{b}GACAGCTAACTATGGCTACACCTTCGGTTCGGGGACCAGGTTAACCGTTGTAG"
        result_inference = igor_model.evaluate(my_seq, align_params=alignment)
        result[a] = result_inference.likelihood
        s += result_inference.likelihood
        print(result_inference.likelihood)
print("Comparison with sum of likelihood: ",  s)

In [ ]:
## comparison between the pgen of an amino-acid sequence 
## and the sum over all corresponding nucleotide sequence

aa_cdr3 = "CARNYEQYF"
Vs = righor.genes_matching('TRBV23', igor_model)
Js = righor.genes_matching('TRBJ2-3', igor_model)

# sum the likelihood of all possible nucleotides sequence for the CDR3
all_nts = list(righor.aa_to_n(aa_cdr3))
s = sum(igor_model.evaluate((righor.Dna(seq), Vs, Js)).likelihood for seq in tqdm(all_nts))

# compute the amino-acid likelihood
result_aa = igor_model.evaluate((righor.AminoAcid(aa_cdr3), Vs, Js))
print(s, result_aa.likelihood)

